In [141]:
import pandas as pd
import numpy as np
import random
import string

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import confusion_matrix, RocCurveDisplay, plot_confusion_matrix, roc_curve, auc, roc_auc_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from numpy import arange, argmax
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree

import matplotlib.pyplot as plt
%matplotlib inline

from numpy import mean

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_colwidth", None)

## Load the data

In [142]:
balances_df = pd.read_csv("balances.csv", sep=";")
collateral_df = pd.read_csv("collateral.csv", sep=";")
customers_df = pd.read_csv("customers.csv", sep=";")


balances_prep = pd.read_csv("balances_prep_simple_v2.csv")
balances_prep.drop("Unnamed: 0", axis=1, inplace=True)

## Explore balances_prep

In [143]:
balances_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39339 entries, 0 to 39338
Data columns (total 65 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   customer_id_                                 39339 non-null  object 
 1   account_id_                                  39339 non-null  object 
 2   open_date_                                   39339 non-null  object 
 3   original_principal_min                       39339 non-null  int64  
 4   original_principal_max                       39339 non-null  int64  
 5   original_principal_std                       39339 non-null  float64
 6   original_principal_max_minus_min             39339 non-null  int64  
 7   reporting_date_count                         39339 non-null  int64  
 8   total_cust_exposure_min                      39339 non-null  int64  
 9   total_cust_exposure_max                      39339 non-null  int64  
 10

In [144]:
def new_labels(row):
    return 0 if row["prepayment_status_unique"] == "['no']" else 1

In [145]:
balances_prep["prepayment_status_unique"] = balances_prep.apply(new_labels, axis=1)

## Explore customers_df

In [146]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34515 entries, 0 to 34514
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   customer_id      34515 non-null  object
 1   age              34515 non-null  object
 2   maritial_status  34515 non-null  object
 3   town             31440 non-null  object
 4   empl_type        34515 non-null  object
 5   profession       34515 non-null  int64 
 6   tot_mnth_income  34515 non-null  int64 
 7   gross_salary     34515 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 2.1+ MB


In [147]:
customers_df.describe()

,profession,tot_mnth_income,gross_salary
count,34515.000000,34515.000000,3.451500e+04
mean,24.811444,2613.860930,2.101830e+03
std,13.471852,9573.384642,7.965116e+03
min,0.000000,0.000000,0.000000e+00
25%,11.000000,994.000000,7.130000e+02
50%,27.000000,1555.000000,1.398000e+03
75%,36.000000,2525.500000,2.411000e+03
max,44.000000,868284.000000,1.116359e+06


In [148]:
customers_df.head()

,customer_id,age,maritial_status,town,empl_type,profession,tot_mnth_income,gross_salary
0,0x966001FD3EF723FC5F,Q251,U67,Sofia,I508,2,5138,6000
1,0xBA04902FFAA73BF01A,Z829,K16,Smolyan,Q237,11,3551,119
2,0xC28B6C6C55416E68A9,Z829,A49,Plovdiv,R412,6,1384,119
3,0x07C6BA3D65466D8A6A,J454,U67,Sofia,I508,41,1652,1128
4,0xF49D6CCED11944AB48,G219,U67,Sofia,I508,10,9655,11225


### Explore columns

#### Age

In [149]:
customers_df["age"].value_counts()

P508    7775
T555    6815
Z829    6409
G219    4898
Q251    3172
A788    2426
J454    1876
H294     766
X693     179
M764     110
F953      54
O456      32
S205       2
W677       1
Name: age, dtype: int64

In [150]:
len(customers_df["age"].value_counts())

14

#### marital_status

In [151]:
customers_df["maritial_status"].value_counts()

K16    15913
U67    12934
A49     3549
M45      764
O32      739
Q37      375
I63      241
Name: maritial_status, dtype: int64

In [152]:
len(customers_df["maritial_status"].value_counts())

7

#### town

Мога да ги сложа в три категории - столица, големи, малки градове

In [153]:
customers_df["town"].value_counts()

Sofia             8958
Plovdiv           4547
Varna             2565
Burgas            2172
Veliko Tarnovo    1699
Stara Zagora      1558
Blagoevgrad       1073
Vratsa            1048
Pleven             945
Ruse               797
Shumen             741
Pazardzhik         655
Haskovo            588
Montana            553
Dobrich            447
Yambol             417
Razgrad            343
Pernik             339
Vidin              327
Gabrovo            311
Lovech             270
Smolyan            257
Silistra           202
Kardzhali          194
Kyustendil         184
Sliven             163
Targovishte         87
Name: town, dtype: int64

In [154]:
len(customers_df["town"].value_counts())

27

разпределението на градове се базира на:
https://bg.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D1%8A%D0%BA_%D0%BD%D0%B0_%D0%B3%D1%80%D0%B0%D0%B4%D0%BE%D0%B2%D0%B5%D1%82%D0%B5_%D0%B2_%D0%91%D1%8A%D0%BB%D0%B3%D0%B0%D1%80%D0%B8%D1%8F_%D0%BF%D0%BE_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5

In [155]:
major_towns = ["Plovdiv", "Varna"]
big_towns = ["Burgas", "Ruse", "Stara Zagora", "Pleven"]
medium_towns = ["Sliven", 
                "Dobrich", 
                "Shumen", 
                "Pernik", 
                "Haskovo", 
                "Yambol", 
                "Pazardzhik", 
                "Blagoevgrad", 
                "Veliko Tarnovo", 
                "Vratsa", 
                "Gabrovo", 
                "Asenovgrad", 
                "Vidin", 
                "Kazanlak", 
                "Kyustendil", 
                "Kardzhali", 
                "Montana", 
                "Dimitrovgrad", 
                "Targovishte", 
                "Lovech", 
                "Silistra", 
                "Razgrad", 
                "Dupnitsa", 
                "Gorna Oryahovitsa", 
                "Smolyan", 
                "Svishtov"]



def new_town_labels(row):
    
#     print(row)
    if row == "Sofia":
        return "Capital"
    elif row in major_towns:
        return "Major Town"
    elif row in big_towns:
        return "Big Town"
    elif row in medium_towns:
        return "MediumS Town"
    else:
        return np.nan

In [157]:
customers_df.loc[:50, ["town", "town_transformed"]]

,town,town_transformed
0,Sofia,Capital
1,Smolyan,MediumS Town
2,Plovdiv,Major Town
3,Sofia,Capital
4,Sofia,Capital
5,Plovdiv,Major Town
6,Stara Zagora,Big Town
7,Veliko Tarnovo,MediumS Town
8,Sofia,Capital
9,Varna,Major Town


#### empl_type

In [158]:
customers_df["empl_type"].value_counts()

I508    30322
Q237     3292
R412      402
U768      124
L320      108
H787      100
Y735       83
C288       45
A350       23
P115       16
Name: empl_type, dtype: int64

In [159]:
len(customers_df["empl_type"].value_counts())

10

### Create dummy_classes

In [160]:
customers_df = customers_df.join(pd.get_dummies(customers_df["maritial_status"], prefix="maritial_status"), how="left")
customers_df = customers_df.join(pd.get_dummies(customers_df["age"], prefix="age"), how="left")
customers_df = customers_df.join(pd.get_dummies(customers_df["empl_type"], prefix="empl_type"), how="left")


In [161]:
customers_df

,customer_id,age,maritial_status,town,empl_type,profession,tot_mnth_income,gross_salary,town_transformed,maritial_status_A49,maritial_status_I63,maritial_status_K16,maritial_status_M45,maritial_status_O32,maritial_status_Q37,maritial_status_U67,age_A788,age_F953,age_G219,age_H294,age_J454,age_M764,age_O456,age_P508,age_Q251,age_S205,age_T555,age_W677,age_X693,age_Z829,empl_type_A350,empl_type_C288,empl_type_H787,empl_type_I508,empl_type_L320,empl_type_P115,empl_type_Q237,empl_type_R412,empl_type_U768,empl_type_Y735
0,0x966001FD3EF723FC5F,Q251,U67,Sofia,I508,2,5138,6000,Capital,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0xBA04902FFAA73BF01A,Z829,K16,Smolyan,Q237,11,3551,119,MediumS Town,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,0xC28B6C6C55416E68A9,Z829,A49,Plovdiv,R412,6,1384,119,Major Town,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
3,0x07C6BA3D65466D8A6A,J454,U67,Sofia,I508,41,1652,1128,Capital,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0xF49D6CCED11944AB48,G219,U67,Sofia,I508,10,9655,11225,Capital,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,0x4FE7E433DDF7160BCB,Z829,U67,Veliko Tarnovo,I508,6,893,1139,MediumS Town,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
34511,0xF760CDB2C6B6DE4997,Z829,K16,Veliko Tarnovo,I508,43,1985,2520,MediumS Town,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
34512,0x0359E3A2B71F02FFC1,H294,U67,Veliko Tarnovo,I508,20,1099,972,MediumS Town,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
34513,0x141C729EC4693C9345,G219,K16,Shumen,Q237,6,53554,119,MediumS Town,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### Fill in missing values (town)

In [162]:
customers_df.columns

Index(['customer_id', 'age', 'maritial_status', 'town', 'empl_type',
       'profession', 'tot_mnth_income', 'gross_salary', 'town_transformed',
       'maritial_status_A49', 'maritial_status_I63', 'maritial_status_K16',
       'maritial_status_M45', 'maritial_status_O32', 'maritial_status_Q37',
       'maritial_status_U67', 'age_A788', 'age_F953', 'age_G219', 'age_H294',
       'age_J454', 'age_M764', 'age_O456', 'age_P508', 'age_Q251', 'age_S205',
       'age_T555', 'age_W677', 'age_X693', 'age_Z829', 'empl_type_A350',
       'empl_type_C288', 'empl_type_H787', 'empl_type_I508', 'empl_type_L320',
       'empl_type_P115', 'empl_type_Q237', 'empl_type_R412', 'empl_type_U768',
       'empl_type_Y735'],
      dtype='object')

In [163]:
customers_df
town_features = ['profession', 'tot_mnth_income', 'gross_salary',
       'maritial_status_A49', 'maritial_status_I63', 'maritial_status_K16',
       'maritial_status_M45', 'maritial_status_O32', 'maritial_status_Q37',
       'maritial_status_U67', 'age_A788', 'age_F953', 'age_G219', 'age_H294',
       'age_J454', 'age_M764', 'age_O456', 'age_P508', 'age_Q251', 'age_S205',
       'age_T555', 'age_W677', 'age_X693', 'age_Z829', 'empl_type_A350',
       'empl_type_C288', 'empl_type_H787', 'empl_type_I508', 'empl_type_L320',
       'empl_type_P115', 'empl_type_Q237', 'empl_type_R412', 'empl_type_U768',
       'empl_type_Y735']

X = customers_df[town_features] 
y = customers_df["town_transformed"]

#### Split on train and test

In [164]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34515 entries, 0 to 34514
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   profession           34515 non-null  int64
 1   tot_mnth_income      34515 non-null  int64
 2   gross_salary         34515 non-null  int64
 3   maritial_status_A49  34515 non-null  uint8
 4   maritial_status_I63  34515 non-null  uint8
 5   maritial_status_K16  34515 non-null  uint8
 6   maritial_status_M45  34515 non-null  uint8
 7   maritial_status_O32  34515 non-null  uint8
 8   maritial_status_Q37  34515 non-null  uint8
 9   maritial_status_U67  34515 non-null  uint8
 10  age_A788             34515 non-null  uint8
 11  age_F953             34515 non-null  uint8
 12  age_G219             34515 non-null  uint8
 13  age_H294             34515 non-null  uint8
 14  age_J454             34515 non-null  uint8
 15  age_M764             34515 non-null  uint8
 16  age_O456             3

In [165]:
empty_towns = y[y.isna()].index

In [166]:
X_test = X.loc[empty_towns, :]
y_test = y[empty_towns]

In [167]:
X_train = X[~X.index.isin(empty_towns)]
y_train = y[~y.index.isin(empty_towns)]

#### Predict y_test

##### Scale the data

In [168]:
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data=X_train_sc, columns=X_train.columns)
y_train_values = y_train.values.ravel()

In [169]:
rf_clf = RandomForestClassifier()

In [170]:
rf_clf.fit(X_train_scaled, y_train_values)

RandomForestClassifier()

In [171]:
X_test_sc = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(data=X_test_sc, columns=X_test.columns)

In [172]:
pred_values = rf_clf.predict(X_test_scaled)

In [173]:
y_test = pd.Series(data=pred_values, name="town_transformed")

In [174]:
y_test.value_counts()

Capital         1194
MediumS Town     837
Major Town       638
Big Town         406
Name: town_transformed, dtype: int64

In [175]:
y_test.index = empty_towns

In [176]:
y_comb = pd.concat([y_train, y_test])

In [177]:
customers_df["town_transformed_full"] = y_comb

In [178]:
customers_df.drop("town_transformed", axis=1, inplace=True)

In [179]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34515 entries, 0 to 34514
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            34515 non-null  object
 1   age                    34515 non-null  object
 2   maritial_status        34515 non-null  object
 3   town                   31440 non-null  object
 4   empl_type              34515 non-null  object
 5   profession             34515 non-null  int64 
 6   tot_mnth_income        34515 non-null  int64 
 7   gross_salary           34515 non-null  int64 
 8   maritial_status_A49    34515 non-null  uint8 
 9   maritial_status_I63    34515 non-null  uint8 
 10  maritial_status_K16    34515 non-null  uint8 
 11  maritial_status_M45    34515 non-null  uint8 
 12  maritial_status_O32    34515 non-null  uint8 
 13  maritial_status_Q37    34515 non-null  uint8 
 14  maritial_status_U67    34515 non-null  uint8 
 15  age_A788           

### Create town dummy

In [180]:
customers_df.rename({"town_transformed_full": "town_transformed"}, axis=1, inplace=True)

In [181]:
customers_df = customers_df.join(pd.get_dummies(customers_df["town_transformed"], prefix="town_transformed"), how="left")

### Drop cat data

In [182]:
customers_df.drop(["age", "maritial_status", "town", "empl_type", "town_transformed"], axis=1, inplace=True)

## Explore collateral

In [183]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   account_id            40794 non-null  object 
 1   collateral_id         40794 non-null  object 
 2   market_value          40794 non-null  int64  
 3   collateral_type       40794 non-null  object 
 4   building_type         40264 non-null  float64
 5   mortgage_utilisation  40774 non-null  float64
 6   town                  40748 non-null  object 
 7   area                  40718 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 2.5+ MB


In [184]:
collateral_df.head()

,account_id,collateral_id,market_value,collateral_type,building_type,mortgage_utilisation,town,area
0,0x41702C36588DDB37F2DD861C5DB40111,0x82164E87B8688CBEC83A21BD4D,136800,M100,1.0,1.0,Plovdiv,84.276
1,0x05354063C1E883849C745C1F1B0418D9,0xA7B4F5E66A9AA942428F810CF8,397800,M101,1.0,1.0,Plovdiv,344.172
2,0x9A6BED3374ABA6453F5BD729DF4E3E01,0x845E56B7B510F20CA2AB7FAB21,95237,M100,2.0,1.0,Sofia,50.448
3,0x480CA3D2B69E387AEE0040B33C0FA21C,0xA069E952426A04A9529D9C011A,98574,M100,1.0,1.0,Plovdiv,116.460
4,0xC6CB9AE910C26A4C142CB6600E8599D6,0xC76BF79BB907B0F68DF52F59FE,187560,M100,1.0,1.0,Plovdiv,161.088


In [185]:
collateral_df["collateral_type"].value_counts()

M100    34668
M101     5517
M105      463
M102       74
M103       71
M104        1
Name: collateral_type, dtype: int64

In [186]:
collateral_df["building_type"].value_counts()

1.0    31275
2.0     8989
Name: building_type, dtype: int64

In [187]:
collateral_df["mortgage_utilisation"].value_counts()

1.0    34308
2.0     6466
Name: mortgage_utilisation, dtype: int64

In [188]:
collateral_df["town"].value_counts()

Sofia             13674
Plovdiv            6323
Varna              3495
Burgas             3086
Veliko Tarnovo     1820
Stara Zagora       1653
Pleven             1091
Blagoevgrad        1014
Vratsa              929
Ruse                872
Shumen              744
Pazardzhik          631
Dobrich             605
Montana             591
Pernik              545
Haskovo             506
Sliven              443
Yambol              439
Razgrad             399
Gabrovo             357
Kyustendil          352
Lovech              262
Targovishte         243
Silistra            186
Vidin               180
Kardzhali           157
Smolyan             151
Name: town, dtype: int64

### Transform town

In [189]:
collateral_df["town_transformed"] = collateral_df["town"].apply(new_town_labels)

### collateral_type

In [190]:
collateral_df = collateral_df.join(pd.get_dummies(collateral_df["collateral_type"], prefix="collateral_type"), how="left")

In [191]:
collateral_df

,account_id,collateral_id,market_value,collateral_type,building_type,mortgage_utilisation,town,area,town_transformed,collateral_type_M100,collateral_type_M101,collateral_type_M102,collateral_type_M103,collateral_type_M104,collateral_type_M105
0,0x41702C36588DDB37F2DD861C5DB40111,0x82164E87B8688CBEC83A21BD4D,136800,M100,1.0,1.0,Plovdiv,84.276,Major Town,1,0,0,0,0,0
1,0x05354063C1E883849C745C1F1B0418D9,0xA7B4F5E66A9AA942428F810CF8,397800,M101,1.0,1.0,Plovdiv,344.172,Major Town,0,1,0,0,0,0
2,0x9A6BED3374ABA6453F5BD729DF4E3E01,0x845E56B7B510F20CA2AB7FAB21,95237,M100,2.0,1.0,Sofia,50.448,Capital,1,0,0,0,0,0
3,0x480CA3D2B69E387AEE0040B33C0FA21C,0xA069E952426A04A9529D9C011A,98574,M100,1.0,1.0,Plovdiv,116.460,Major Town,1,0,0,0,0,0
4,0xC6CB9AE910C26A4C142CB6600E8599D6,0xC76BF79BB907B0F68DF52F59FE,187560,M100,1.0,1.0,Plovdiv,161.088,Major Town,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40789,0x8B3D24EFFF236A42A7AAECE4D8CDD2E7,0x9F402A07EB714CC16E8AB9662D,82800,M100,1.0,1.0,Veliko Tarnovo,85.860,MediumS Town,1,0,0,0,0,0
40790,0x0C16AB8C478403130D0F169F46D4F037,0xE7F02EC0D2E5A810C81F62CF22,134118,M100,1.0,1.0,Veliko Tarnovo,107.532,MediumS Town,1,0,0,0,0,0
40791,0x48782A1FF9A5B9EC94E2FD7112DB151D,0x5142E86E6D9D6520A4A1F75C34,106320,M101,1.0,1.0,Lovech,148.800,MediumS Town,0,1,0,0,0,0
40792,0x60E568486E4464F8BE16472EF62BBCCE,0x674AF0BF63809C37B5BD9F3C34,132636,M100,1.0,1.0,Veliko Tarnovo,108.720,MediumS Town,1,0,0,0,0,0


### Fill in missing values

In [192]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   account_id            40794 non-null  object 
 1   collateral_id         40794 non-null  object 
 2   market_value          40794 non-null  int64  
 3   collateral_type       40794 non-null  object 
 4   building_type         40264 non-null  float64
 5   mortgage_utilisation  40774 non-null  float64
 6   town                  40748 non-null  object 
 7   area                  40718 non-null  float64
 8   town_transformed      40748 non-null  object 
 9   collateral_type_M100  40794 non-null  uint8  
 10  collateral_type_M101  40794 non-null  uint8  
 11  collateral_type_M102  40794 non-null  uint8  
 12  collateral_type_M103  40794 non-null  uint8  
 13  collateral_type_M104  40794 non-null  uint8  
 14  collateral_type_M105  40794 non-null  uint8  
dtypes: float64(3), int6

In [193]:
collateral_df.columns

Index(['account_id', 'collateral_id', 'market_value', 'collateral_type',
       'building_type', 'mortgage_utilisation', 'town', 'area',
       'town_transformed', 'collateral_type_M100', 'collateral_type_M101',
       'collateral_type_M102', 'collateral_type_M103', 'collateral_type_M104',
       'collateral_type_M105'],
      dtype='object')

In [194]:
town_tr_features = ['market_value', 'collateral_type_M100', 'collateral_type_M101',
       'collateral_type_M102', 'collateral_type_M103', 'collateral_type_M104',
       'collateral_type_M105']

X = collateral_df[town_tr_features] 
y = collateral_df["town_transformed"]

#### Split on train and test

In [195]:
empty_towns = y[y.isna()].index

In [196]:
X_test = X.loc[empty_towns, :]
y_test = y[empty_towns]

In [197]:
X_train = X[~X.index.isin(empty_towns)]
y_train = y[~y.index.isin(empty_towns)]

#### Predict y_test

##### Scale the data

In [198]:
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data=X_train_sc, columns=X_train.columns)
y_train_values = y_train.values.ravel()

In [199]:
rf_clf = RandomForestClassifier()

In [200]:
rf_clf.fit(X_train_scaled, y_train_values)

RandomForestClassifier()

In [201]:
X_test_sc = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(data=X_test_sc, columns=X_test.columns)

In [202]:
pred_values = rf_clf.predict(X_test_scaled)

In [203]:
y_test = pd.Series(data=pred_values, name="town_transformed")

In [204]:
y_test.value_counts()

Capital         32
Major Town       9
Big Town         3
MediumS Town     2
Name: town_transformed, dtype: int64

In [205]:
y_test.index = empty_towns

In [206]:
y_comb = pd.concat([y_train, y_test])

In [207]:
collateral_df["town_transformed_full"] = y_comb

In [208]:
collateral_df.drop("town_transformed", axis=1, inplace=True)

In [209]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   account_id             40794 non-null  object 
 1   collateral_id          40794 non-null  object 
 2   market_value           40794 non-null  int64  
 3   collateral_type        40794 non-null  object 
 4   building_type          40264 non-null  float64
 5   mortgage_utilisation   40774 non-null  float64
 6   town                   40748 non-null  object 
 7   area                   40718 non-null  float64
 8   collateral_type_M100   40794 non-null  uint8  
 9   collateral_type_M101   40794 non-null  uint8  
 10  collateral_type_M102   40794 non-null  uint8  
 11  collateral_type_M103   40794 non-null  uint8  
 12  collateral_type_M104   40794 non-null  uint8  
 13  collateral_type_M105   40794 non-null  uint8  
 14  town_transformed_full  40794 non-null  object 
dtypes:

In [210]:
collateral_df.drop("collateral_type", axis=1, inplace=True)

#### Create town dummies

In [211]:
### Create town dummy

collateral_df.rename({"town_transformed_full": "town_transformed"}, axis=1, inplace=True)

collateral_df = collateral_df.join(pd.get_dummies(collateral_df["town_transformed"], prefix="town_transformed"), how="left")

collateral_df.drop("town", axis=1, inplace=True)

In [212]:
collateral_df.drop("town_transformed", axis=1, inplace=True)

#### fill in building_type

In [213]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_id                     40794 non-null  object 
 1   collateral_id                  40794 non-null  object 
 2   market_value                   40794 non-null  int64  
 3   building_type                  40264 non-null  float64
 4   mortgage_utilisation           40774 non-null  float64
 5   area                           40718 non-null  float64
 6   collateral_type_M100           40794 non-null  uint8  
 7   collateral_type_M101           40794 non-null  uint8  
 8   collateral_type_M102           40794 non-null  uint8  
 9   collateral_type_M103           40794 non-null  uint8  
 10  collateral_type_M104           40794 non-null  uint8  
 11  collateral_type_M105           40794 non-null  uint8  
 12  town_transformed_Big Town      40794 non-null 

In [214]:
collateral_df.columns

Index(['account_id', 'collateral_id', 'market_value', 'building_type',
       'mortgage_utilisation', 'area', 'collateral_type_M100',
       'collateral_type_M101', 'collateral_type_M102', 'collateral_type_M103',
       'collateral_type_M104', 'collateral_type_M105',
       'town_transformed_Big Town', 'town_transformed_Capital',
       'town_transformed_Major Town', 'town_transformed_MediumS Town'],
      dtype='object')

In [215]:
building_type_features = ['market_value', 'collateral_type_M100', 'collateral_type_M101',
       'collateral_type_M102', 'collateral_type_M103', 'collateral_type_M104',
       'collateral_type_M105', 'town_transformed_Big Town', 'town_transformed_Capital',
       'town_transformed_Major Town', 'town_transformed_MediumS Town']

X = collateral_df[building_type_features] 
y = collateral_df["building_type"]

#### Split on train and test

In [216]:
empty_towns = y[y.isna()].index

In [217]:
X_test = X.loc[empty_towns, :]
y_test = y[empty_towns]

In [218]:
X_train = X[~X.index.isin(empty_towns)]
y_train = y[~y.index.isin(empty_towns)]

#### Predict y_test

##### Scale the data

In [219]:
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data=X_train_sc, columns=X_train.columns)
y_train_values = y_train.values.ravel()

In [220]:
rf_clf = RandomForestClassifier()

In [221]:
rf_clf.fit(X_train_scaled, y_train_values)

RandomForestClassifier()

In [222]:
X_test_sc = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(data=X_test_sc, columns=X_test.columns)

In [223]:
pred_values = rf_clf.predict(X_test_scaled)

In [224]:
y_test = pd.Series(data=pred_values, name="building_type")

In [225]:
y_test.value_counts()

1.0    412
2.0    118
Name: building_type, dtype: int64

In [226]:
y_test.index = empty_towns

In [227]:
y_comb = pd.concat([y_train, y_test])

In [228]:
collateral_df["building_type_full"] = y_comb

In [229]:
collateral_df.drop("building_type", axis=1, inplace=True)

In [230]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_id                     40794 non-null  object 
 1   collateral_id                  40794 non-null  object 
 2   market_value                   40794 non-null  int64  
 3   mortgage_utilisation           40774 non-null  float64
 4   area                           40718 non-null  float64
 5   collateral_type_M100           40794 non-null  uint8  
 6   collateral_type_M101           40794 non-null  uint8  
 7   collateral_type_M102           40794 non-null  uint8  
 8   collateral_type_M103           40794 non-null  uint8  
 9   collateral_type_M104           40794 non-null  uint8  
 10  collateral_type_M105           40794 non-null  uint8  
 11  town_transformed_Big Town      40794 non-null  uint8  
 12  town_transformed_Capital       40794 non-null 

#### Fill in mortgage utilization

In [231]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_id                     40794 non-null  object 
 1   collateral_id                  40794 non-null  object 
 2   market_value                   40794 non-null  int64  
 3   mortgage_utilisation           40774 non-null  float64
 4   area                           40718 non-null  float64
 5   collateral_type_M100           40794 non-null  uint8  
 6   collateral_type_M101           40794 non-null  uint8  
 7   collateral_type_M102           40794 non-null  uint8  
 8   collateral_type_M103           40794 non-null  uint8  
 9   collateral_type_M104           40794 non-null  uint8  
 10  collateral_type_M105           40794 non-null  uint8  
 11  town_transformed_Big Town      40794 non-null  uint8  
 12  town_transformed_Capital       40794 non-null 

In [232]:
collateral_df.columns

Index(['account_id', 'collateral_id', 'market_value', 'mortgage_utilisation',
       'area', 'collateral_type_M100', 'collateral_type_M101',
       'collateral_type_M102', 'collateral_type_M103', 'collateral_type_M104',
       'collateral_type_M105', 'town_transformed_Big Town',
       'town_transformed_Capital', 'town_transformed_Major Town',
       'town_transformed_MediumS Town', 'building_type_full'],
      dtype='object')

In [233]:
mortgage_utilisation_features = ['market_value', 'collateral_type_M100', 'collateral_type_M101',
       'collateral_type_M102', 'collateral_type_M103', 'collateral_type_M104',
       'collateral_type_M105', 'town_transformed_Big Town', 'town_transformed_Capital',
       'town_transformed_Major Town', 'town_transformed_MediumS Town',
       'building_type_full']

X = collateral_df[mortgage_utilisation_features] 
y = collateral_df["mortgage_utilisation"]

#### Split on train and test

In [234]:
empty_towns = y[y.isna()].index

In [235]:
X_test = X.loc[empty_towns, :]
y_test = y[empty_towns]

In [236]:
X_train = X[~X.index.isin(empty_towns)]
y_train = y[~y.index.isin(empty_towns)]

#### Predict y_test

##### Scale the data

In [237]:
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data=X_train_sc, columns=X_train.columns)
y_train_values = y_train.values.ravel()

In [238]:
rf_clf = RandomForestClassifier()

In [239]:
rf_clf.fit(X_train_scaled, y_train_values)

RandomForestClassifier()

In [240]:
X_test_sc = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(data=X_test_sc, columns=X_test.columns)

In [241]:
pred_values = rf_clf.predict(X_test_scaled)

In [242]:
y_test = pd.Series(data=pred_values, name="mortgage_utilisation")

In [243]:
y_test.value_counts()

1.0    20
Name: mortgage_utilisation, dtype: int64

In [244]:
y_test.index = empty_towns

In [245]:
y_comb = pd.concat([y_train, y_test])

In [246]:
collateral_df["mortgage_utilisation_full"] = y_comb

In [247]:
collateral_df.drop("mortgage_utilisation", axis=1, inplace=True)

In [248]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_id                     40794 non-null  object 
 1   collateral_id                  40794 non-null  object 
 2   market_value                   40794 non-null  int64  
 3   area                           40718 non-null  float64
 4   collateral_type_M100           40794 non-null  uint8  
 5   collateral_type_M101           40794 non-null  uint8  
 6   collateral_type_M102           40794 non-null  uint8  
 7   collateral_type_M103           40794 non-null  uint8  
 8   collateral_type_M104           40794 non-null  uint8  
 9   collateral_type_M105           40794 non-null  uint8  
 10  town_transformed_Big Town      40794 non-null  uint8  
 11  town_transformed_Capital       40794 non-null  uint8  
 12  town_transformed_Major Town    40794 non-null 

#### Fill in area

In [249]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_id                     40794 non-null  object 
 1   collateral_id                  40794 non-null  object 
 2   market_value                   40794 non-null  int64  
 3   area                           40718 non-null  float64
 4   collateral_type_M100           40794 non-null  uint8  
 5   collateral_type_M101           40794 non-null  uint8  
 6   collateral_type_M102           40794 non-null  uint8  
 7   collateral_type_M103           40794 non-null  uint8  
 8   collateral_type_M104           40794 non-null  uint8  
 9   collateral_type_M105           40794 non-null  uint8  
 10  town_transformed_Big Town      40794 non-null  uint8  
 11  town_transformed_Capital       40794 non-null  uint8  
 12  town_transformed_Major Town    40794 non-null 

In [250]:
collateral_df.columns

Index(['account_id', 'collateral_id', 'market_value', 'area',
       'collateral_type_M100', 'collateral_type_M101', 'collateral_type_M102',
       'collateral_type_M103', 'collateral_type_M104', 'collateral_type_M105',
       'town_transformed_Big Town', 'town_transformed_Capital',
       'town_transformed_Major Town', 'town_transformed_MediumS Town',
       'building_type_full', 'mortgage_utilisation_full'],
      dtype='object')

In [251]:
area_features = ['market_value', 'collateral_type_M100', 'collateral_type_M101',
       'collateral_type_M102', 'collateral_type_M103', 'collateral_type_M104',
       'collateral_type_M105', 'town_transformed_Big Town', 'town_transformed_Capital',
       'town_transformed_Major Town', 'town_transformed_MediumS Town',
       'building_type_full', 'mortgage_utilisation_full']

X = collateral_df[area_features] 
y = collateral_df["area"]

#### Split on train and test

In [252]:
empty_towns = y[y.isna()].index

In [253]:
X_test = X.loc[empty_towns, :]
y_test = y[empty_towns]

In [254]:
X_train = X[~X.index.isin(empty_towns)]
y_train = y[~y.index.isin(empty_towns)]

#### Predict y_test

##### Scale the data

In [255]:
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data=X_train_sc, columns=X_train.columns)
y_train_values = y_train.values.ravel()

In [256]:
rf_clf = RandomForestRegressor()

In [257]:
rf_clf.fit(X_train_scaled, y_train_values)

RandomForestRegressor()

In [258]:
X_test_sc = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(data=X_test_sc, columns=X_test.columns)

In [259]:
pred_values = rf_clf.predict(X_test_scaled)

In [260]:
y_test = pd.Series(data=pred_values, name="area")

In [261]:
y_test.value_counts()

110.685760    1
76.446070     1
96.166720     1
70.342224     1
116.489280    1
71.042280     1
88.001697     1
66.256822     1
63.712512     1
133.014480    1
64.622831     1
108.922440    1
360.553714    1
77.039453     1
86.540396     1
187.548640    1
255.164520    1
162.975600    1
104.467936    1
126.358228    1
339.376920    1
76.181880     1
152.407069    1
71.236460     1
86.890688     1
71.123923     1
81.400862     1
55.383240     1
106.963632    1
118.502970    1
74.612980     1
87.513796     1
126.938040    1
52.021464     1
160.364688    1
111.564833    1
102.990480    1
104.565270    1
74.936325     1
84.996051     1
76.209600     1
56.875080     1
153.865600    1
80.158252     1
133.577240    1
74.175739     1
206.258440    1
103.966856    1
45.161218     1
87.552000     1
73.212912     1
63.032124     1
76.596000     1
70.031712     1
109.127744    1
75.868464     1
82.169729     1
147.599724    1
121.520237    1
94.996920     1
76.334400     1
950.882500    1
77.36537

In [262]:
y_test.index = empty_towns

In [263]:
y_comb = pd.concat([y_train, y_test])

In [264]:
collateral_df["area_full"] = y_comb

In [265]:
collateral_df.drop("area", axis=1, inplace=True)

In [266]:
collateral_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40794 entries, 0 to 40793
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_id                     40794 non-null  object 
 1   collateral_id                  40794 non-null  object 
 2   market_value                   40794 non-null  int64  
 3   collateral_type_M100           40794 non-null  uint8  
 4   collateral_type_M101           40794 non-null  uint8  
 5   collateral_type_M102           40794 non-null  uint8  
 6   collateral_type_M103           40794 non-null  uint8  
 7   collateral_type_M104           40794 non-null  uint8  
 8   collateral_type_M105           40794 non-null  uint8  
 9   town_transformed_Big Town      40794 non-null  uint8  
 10  town_transformed_Capital       40794 non-null  uint8  
 11  town_transformed_Major Town    40794 non-null  uint8  
 12  town_transformed_MediumS Town  40794 non-null 

## Merge all data

In [267]:
customers_df.columns

Index(['customer_id', 'profession', 'tot_mnth_income', 'gross_salary',
       'maritial_status_A49', 'maritial_status_I63', 'maritial_status_K16',
       'maritial_status_M45', 'maritial_status_O32', 'maritial_status_Q37',
       'maritial_status_U67', 'age_A788', 'age_F953', 'age_G219', 'age_H294',
       'age_J454', 'age_M764', 'age_O456', 'age_P508', 'age_Q251', 'age_S205',
       'age_T555', 'age_W677', 'age_X693', 'age_Z829', 'empl_type_A350',
       'empl_type_C288', 'empl_type_H787', 'empl_type_I508', 'empl_type_L320',
       'empl_type_P115', 'empl_type_Q237', 'empl_type_R412', 'empl_type_U768',
       'empl_type_Y735', 'town_transformed_Big Town',
       'town_transformed_Capital', 'town_transformed_Major Town',
       'town_transformed_MediumS Town'],
      dtype='object')

In [268]:
balances_prep.rename(columns={"customer_id_":"customer_id",
                             "account_id_": "account_id"}, inplace=True)

In [269]:
cust_bal = pd.merge(balances_prep, customers_df, on="customer_id", how="left")
all_files = pd.merge(cust_bal, collateral_df, on="account_id", how="left")

In [270]:
all_files.drop(["town_transformed_Big Town_x", 
                "town_transformed_Capital_x", 
                "town_transformed_Major Town_x", 
                "town_transformed_MediumS Town_x"], axis=1, inplace=True)

In [271]:
all_files.rename({"town_transformed_Big Town_y": "town_Big",
                 "town_transformed_Capital_y": "town_Capital",
                 "town_transformed_Major Town_y": "town_Major",
                 "town_transformed_MediumS Town_y": "town_Medium_Size"}, axis=1, inplace=True)

In [272]:
all_files.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40602 entries, 0 to 40601
Data columns (total 114 columns):
 #    Column                                       Non-Null Count  Dtype  
---   ------                                       --------------  -----  
 0    customer_id                                  40602 non-null  object 
 1    account_id                                   40602 non-null  object 
 2    open_date_                                   40602 non-null  object 
 3    original_principal_min                       40602 non-null  int64  
 4    original_principal_max                       40602 non-null  int64  
 5    original_principal_std                       40602 non-null  float64
 6    original_principal_max_minus_min             40602 non-null  int64  
 7    reporting_date_count                         40602 non-null  int64  
 8    total_cust_exposure_min                      40602 non-null  int64  
 9    total_cust_exposure_max                      40602 non-null

## Fill in rest of data

In [273]:
def feature_fill(df, features_pred_list, target_col, model):
    
    
    
    print("Column: ", col)
    X = df[features_pred_list] 
    y = df[target_col]
    
    
    empty_col_idx = y[y.isna()].index
    
    X_test = X.loc[empty_col_idx, :]
    y_test = y[empty_col_idx]
    
    X_train = X[~X.index.isin(empty_col_idx)]
    y_train = y[~y.index.isin(empty_col_idx)]
    
    
    print("SCALING THE DATA...")
    scaler = StandardScaler()

    X_train_sc = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(data=X_train_sc, columns=X_train.columns)
    y_train_values = y_train.values.ravel()
    
    
    
    print("FITTING MODEL...")
    model.fit(X_train_scaled, y_train_values)
    
    X_test_sc = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(data=X_test_sc, columns=X_test.columns)
    
    pred_values = model.predict(X_test_scaled)
    y_test = pd.Series(data=pred_values, name=target_col)
    
    y_test.index = empty_col_idx
    
    y_comb = pd.concat([y_train, y_test])
    
    col_name_full = target_col + "_full"
    df[col_name_full] = y_comb
    
    df.drop(target_col, axis=1, inplace=True)
    print("COLUMN SUCCESSFULLY PROCESSED")
    print()

In [274]:
all_files.head()

,customer_id,account_id,open_date_,original_principal_min,original_principal_max,original_principal_std,original_principal_max_minus_min,reporting_date_count,total_cust_exposure_min,total_cust_exposure_max,overdraft_min,overdraft_max,overdraft_size,overdraft_count0,overdraft_count1,overdraft_count2plus,consumer_loan_min,consumer_loan_max,consumer_loan_size,consumer_loan_count0,consumer_loan_count1,consumer_loan_count2plus,credit_card_min,credit_card_max,credit_card_size,credit_card_count0,credit_card_count1,credit_card_count2plus,customer_rating_min,customer_rating_max,customer_rating_modeX,maturity_date_min,maturity_date_max,maturity_date_max_minus_min,prepayment_status_unique,l_period_min,l_period_max,l_period_max_minus_min,interest_rate_min,interest_rate_max,interest_rate_max_minus_min,interest_rate_modeX,exp_monthly_payments_min,exp_monthly_payments_max,exp_monthly_payments_std,exp_monthly_payments_max_minus_min,current_principal_min,current_principal_max,current_principal_max_minus_min,interest_payment_min,interest_payment_max,interest_payment_max_minus_min,principal_payment_min,principal_payment_max,principal_payment_max_minus_min,deducted_principal_min,deducted_principal_max,deducted_principal_max_minus_min,deducted_vs_principal_payment_min,deducted_vs_principal_payment_max,deducted_vs_principal_payment_mean,deducted_vs_principal_payment_max_minus_min,principal_left_pctg_min,principal_left_pctg_max,principal_left_pctg_max_minus_min,profession,tot_mnth_income,gross_salary,maritial_status_A49,maritial_status_I63,maritial_status_K16,maritial_status_M45,maritial_status_O32,maritial_status_Q37,maritial_status_U67,age_A788,age_F953,age_G219,age_H294,age_J454,age_M764,age_O456,age_P508,age_Q251,age_S205,age_T555,age_W677,age_X693,age_Z829,empl_type_A350,empl_type_C288,empl_type_H787,empl_type_I508,empl_type_L320,empl_type_P115,empl_type_Q237,empl_type_R412,empl_type_U768,empl_type_Y735,collateral_id,market_value,collateral_type_M100,collateral_type_M101,collateral_type_M102,collateral_type_M103,collateral_type_M104,collateral_type_M105,town_Big,town_Capital,town_Major,town_Medium_Size,building_type_full,mortgage_utilisation_full,area_full
0,0x000100F424893C746F,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,64800,64800,0.0,0,44,56155,63355,0,0,44,44,0,0,0,0,44,44,0,0,0,0,44,44,0,0,3.0,5.5,3.5,2042-06-29,2042-06-29,0,0,24,24,0,3.70,3.70,0.0,3.70,339.819283,339.819283,0.0,0.0,56155,62342,6187,173.144583,192.221167,19.076583,147.598116,166.674700,19.076583,0.0,2458.0,2458.0,-164.251200,2310.401884,39.390899,2474.653083,87.0,96.0,9.0,21.0,660.0,888.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0xA1224902A54E10483EBEEFC973,72360,1,0,0,0,0,0,0,0,0,1,1.0,1.0,47.568
1,0x000224F02B4B39DDFE,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,74880,74880,0.0,0,11,70449,72710,0,0,11,11,0,0,0,0,11,11,0,0,0,0,11,11,0,0,4.0,5.5,4.0,2042-08-16,2042-08-16,0,1,25,25,0,3.90,3.90,0.0,3.90,391.121433,391.121433,0.0,0.0,69729,71273,1544,226.619250,231.637250,5.018000,159.484183,164.502183,5.018000,147.0,3607.0,3460.0,-12.961933,3447.515817,306.287522,3460.477750,93.0,95.0,2.0,42.0,1487.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0x5A1DD802DAC694B446F8C6161B,93600,1,0,0,0,0,0,1,0,0,0,1.0,1.0,81.180
2,0x00025DA4C145ED8EBF,0x8C794E5D7336C4FE59B85E4400973723,2022-04-13,54000,54000,0.0,0,3,53248,54000,0,0,3,3,0,0,0,0,3,3,0,0,0,0,3,3,0,0,5.5,5.5,5.5,2033-04-13,2033-04-13,0,0,11,11,0,2.48,2.48,0.0,2.48,467.844781,467.844781,0.0,0.0,53248,54000,752,110.045867,111.600000,1.554133,356.244781,357.798915,1.554133,0.0,398.0,398.0,-356.244781,40.932685,-106.370337,397.177467,99.0,100.0,1.0,13.0,1663.0,780.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0xD744613939B2BCC38B45EC917E,80400,1,0,0,0,0,0,0,0,0,1,2.0,1.0,74.904
3,0x0003

In [275]:
cols_to_fill_classifier = ['profession',
       'maritial_status_A49', 'maritial_status_I63', 'maritial_status_K16',
       'maritial_status_M45', 'maritial_status_O32', 'maritial_status_Q37',
       'maritial_status_U67', 'age_A788', 'age_F953', 'age_G219', 'age_H294',
       'age_J454', 'age_M764', 'age_O456', 'age_P508', 'age_Q251', 'age_S205',
       'age_T555', 'age_W677', 'age_X693', 'age_Z829', 'empl_type_A350',
       'empl_type_C288', 'empl_type_H787', 'empl_type_I508', 'empl_type_L320',
       'empl_type_P115', 'empl_type_Q237', 'empl_type_R412', 'empl_type_U768',
       'empl_type_Y735']

cols_to_fill_regressor = ['tot_mnth_income', 'gross_salary']

In [276]:
predictors_list = ['original_principal_min',
       'original_principal_max', 'original_principal_std',
       'original_principal_max_minus_min', 'reporting_date_count',
       'total_cust_exposure_min', 'total_cust_exposure_max', 'overdraft_min',
       'overdraft_max', 'overdraft_size', 'overdraft_count0',
       'overdraft_count1', 'overdraft_count2plus', 'consumer_loan_min',
       'consumer_loan_max', 'consumer_loan_size', 'consumer_loan_count0',
       'consumer_loan_count1', 'consumer_loan_count2plus', 'credit_card_min',
       'credit_card_max', 'credit_card_size', 'credit_card_count0',
       'credit_card_count1', 'credit_card_count2plus', 'customer_rating_min',
       'customer_rating_max', 'customer_rating_modeX', 'maturity_date_max_minus_min',
       'prepayment_status_unique', 'l_period_min', 'l_period_max',
       'l_period_max_minus_min', 'interest_rate_min', 'interest_rate_max',
       'interest_rate_max_minus_min', 'interest_rate_modeX',
       'exp_monthly_payments_min', 'exp_monthly_payments_max',
       'exp_monthly_payments_std', 'exp_monthly_payments_max_minus_min',
       'current_principal_min', 'current_principal_max',
       'current_principal_max_minus_min', 'interest_payment_min',
       'interest_payment_max', 'interest_payment_max_minus_min',
       'principal_payment_min', 'principal_payment_max',
       'principal_payment_max_minus_min', 'deducted_principal_min',
       'deducted_principal_max', 'deducted_principal_max_minus_min',
       'deducted_vs_principal_payment_min',
       'deducted_vs_principal_payment_max',
       'deducted_vs_principal_payment_mean',
       'deducted_vs_principal_payment_max_minus_min',
       'principal_left_pctg_min', 'principal_left_pctg_max',
       'principal_left_pctg_max_minus_min',
                  'market_value', 'collateral_type_M100',
       'collateral_type_M101', 'collateral_type_M102', 'collateral_type_M103',
       'collateral_type_M104', 'collateral_type_M105',
       'town_Big', 'town_Capital',
       'town_Major', 'town_Medium_Size',
       'building_type_full', 'mortgage_utilisation_full', 'area_full']

In [277]:
model = RandomForestRegressor()



for col in cols_to_fill_regressor:
    
    feature_fill(df=all_files, features_pred_list=predictors_list, target_col=col, model=model)

Column:  tot_mnth_income
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  gross_salary
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED



In [278]:
all_files.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40602 entries, 0 to 40601
Data columns (total 114 columns):
 #    Column                                       Non-Null Count  Dtype  
---   ------                                       --------------  -----  
 0    customer_id                                  40602 non-null  object 
 1    account_id                                   40602 non-null  object 
 2    open_date_                                   40602 non-null  object 
 3    original_principal_min                       40602 non-null  int64  
 4    original_principal_max                       40602 non-null  int64  
 5    original_principal_std                       40602 non-null  float64
 6    original_principal_max_minus_min             40602 non-null  int64  
 7    reporting_date_count                         40602 non-null  int64  
 8    total_cust_exposure_min                      40602 non-null  int64  
 9    total_cust_exposure_max                      40602 non-null

In [279]:
all_files.head()

,customer_id,account_id,open_date_,original_principal_min,original_principal_max,original_principal_std,original_principal_max_minus_min,reporting_date_count,total_cust_exposure_min,total_cust_exposure_max,overdraft_min,overdraft_max,overdraft_size,overdraft_count0,overdraft_count1,overdraft_count2plus,consumer_loan_min,consumer_loan_max,consumer_loan_size,consumer_loan_count0,consumer_loan_count1,consumer_loan_count2plus,credit_card_min,credit_card_max,credit_card_size,credit_card_count0,credit_card_count1,credit_card_count2plus,customer_rating_min,customer_rating_max,customer_rating_modeX,maturity_date_min,maturity_date_max,maturity_date_max_minus_min,prepayment_status_unique,l_period_min,l_period_max,l_period_max_minus_min,interest_rate_min,interest_rate_max,interest_rate_max_minus_min,interest_rate_modeX,exp_monthly_payments_min,exp_monthly_payments_max,exp_monthly_payments_std,exp_monthly_payments_max_minus_min,current_principal_min,current_principal_max,current_principal_max_minus_min,interest_payment_min,interest_payment_max,interest_payment_max_minus_min,principal_payment_min,principal_payment_max,principal_payment_max_minus_min,deducted_principal_min,deducted_principal_max,deducted_principal_max_minus_min,deducted_vs_principal_payment_min,deducted_vs_principal_payment_max,deducted_vs_principal_payment_mean,deducted_vs_principal_payment_max_minus_min,principal_left_pctg_min,principal_left_pctg_max,principal_left_pctg_max_minus_min,profession,maritial_status_A49,maritial_status_I63,maritial_status_K16,maritial_status_M45,maritial_status_O32,maritial_status_Q37,maritial_status_U67,age_A788,age_F953,age_G219,age_H294,age_J454,age_M764,age_O456,age_P508,age_Q251,age_S205,age_T555,age_W677,age_X693,age_Z829,empl_type_A350,empl_type_C288,empl_type_H787,empl_type_I508,empl_type_L320,empl_type_P115,empl_type_Q237,empl_type_R412,empl_type_U768,empl_type_Y735,collateral_id,market_value,collateral_type_M100,collateral_type_M101,collateral_type_M102,collateral_type_M103,collateral_type_M104,collateral_type_M105,town_Big,town_Capital,town_Major,town_Medium_Size,building_type_full,mortgage_utilisation_full,area_full,tot_mnth_income_full,gross_salary_full
0,0x000100F424893C746F,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,64800,64800,0.0,0,44,56155,63355,0,0,44,44,0,0,0,0,44,44,0,0,0,0,44,44,0,0,3.0,5.5,3.5,2042-06-29,2042-06-29,0,0,24,24,0,3.70,3.70,0.0,3.70,339.819283,339.819283,0.0,0.0,56155,62342,6187,173.144583,192.221167,19.076583,147.598116,166.674700,19.076583,0.0,2458.0,2458.0,-164.251200,2310.401884,39.390899,2474.653083,87.0,96.0,9.0,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0xA1224902A54E10483EBEEFC973,72360,1,0,0,0,0,0,0,0,0,1,1.0,1.0,47.568,660.0,888.0
1,0x000224F02B4B39DDFE,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,74880,74880,0.0,0,11,70449,72710,0,0,11,11,0,0,0,0,11,11,0,0,0,0,11,11,0,0,4.0,5.5,4.0,2042-08-16,2042-08-16,0,1,25,25,0,3.90,3.90,0.0,3.90,391.121433,391.121433,0.0,0.0,69729,71273,1544,226.619250,231.637250,5.018000,159.484183,164.502183,5.018000,147.0,3607.0,3460.0,-12.961933,3447.515817,306.287522,3460.477750,93.0,95.0,2.0,42.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0x5A1DD802DAC694B446F8C6161B,93600,1,0,0,0,0,0,1,0,0,0,1.0,1.0,81.180,1487.0,0.0
2,0x00025DA4C145ED8EBF,0x8C794E5D7336C4FE59B85E4400973723,2022-04-13,54000,54000,0.0,0,3,53248,54000,0,0,3,3,0,0,0,0,3,3,0,0,0,0,3,3,0,0,5.5,5.5,5.5,2033-04-13,2033-04-13,0,0,11,11,0,2.48,2.48,0.0,2.48,467.844781,467.844781,0.0,0.0,53248,54000,752,110.045867,111.600000,1.554133,356.244781,357.798915,1.554133,0.0,398.0,398.0,-356.244781,40.932685,-106.370337,397.177467,99.0,100.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0xD744613939B2BCC38B45EC917E,80400,1,0,0,0,0,0,0,0,0,1,2.0,1.0,74.904,1663.0,780.

In [280]:
model = RandomForestClassifier()



for col in cols_to_fill_classifier:
    
    feature_fill(df=all_files, features_pred_list=predictors_list, target_col=col, model=model)

Column:  profession
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  maritial_status_A49
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  maritial_status_I63
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  maritial_status_K16
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  maritial_status_M45
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  maritial_status_O32
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  maritial_status_Q37
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  maritial_status_U67
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  age_A788
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  age_F953
SCALING THE DATA...
FITTING MODEL...
COLUMN SUCCESSFULLY PROCESSED

Column:  age_G219
SCALING THE DATA...
FITTING MODEL...
COLUMN

## Add New Cols

In [281]:
all_files["salary_principal_ratio"] = all_files["gross_salary_full"] / all_files["current_principal_max"]
all_files["salary_total_cust_exposure_ratio"] = all_files["gross_salary_full"] / all_files["total_cust_exposure_max"]

all_files["tot_income_principal_ratio"] = all_files["tot_mnth_income_full"] / all_files["current_principal_max"]
all_files["tot_income_cust_exposure_ratio"] = all_files["tot_mnth_income_full"] / all_files["total_cust_exposure_max"]

all_files["salary_tot_income_ratio"] = all_files["gross_salary_full"] / all_files["total_cust_exposure_max"]


## Save the data

In [282]:
all_files.to_csv("all_files_v2__.csv")

In [283]:
all_files.shape

(40602, 119)

In [284]:
sample_all = all_files.sample(frac=0.33, replace=False, random_state=1)

In [285]:
sample_all.to_csv("sample_all_v2__.csv")